In [1]:
from skimage.measure import label, regionprops
import math

from utils.DiceLoss import DiceLoss
from utils.JaccardIndex import JaccardIndex
import glob
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Incarcare imagini presegmentate
seg_folder = 'G:/MachineLearning/liver-database/validation/'
models = ['DenseNet201', 'Inception', 'ResNet152', 'ResNeXt101']
# models = ['DenseNet201']
# models = ['UNet', 'Fusion']
seg = np.zeros((len(models), 4776, 512, 512), dtype=bool)
seg_proc = np.zeros((len(models), 4776, 512, 512), dtype=bool)
for i in range(len(models)):
    full_path = seg_folder + models[i]
    seg_files = glob.glob(full_path + '/*.ct')
    for j in range(len(seg_files)):
        seg[i, j, :, :] = np.fromfile(seg_files[j], dtype=np.int8).squeeze().reshape([512, 512]).astype(bool)
    print(f"Done loading {models[i]}")

seg = torch.tensor(seg)

Done loading DenseNet201
Done loading Inception
Done loading ResNet152
Done loading ResNeXt101


In [3]:
for i in range(len(models)):
    print(f"----------------- {models[i]} -----------------")
    for j in range(4776):
        slice = np.copy(np.squeeze(seg[i, j, :, :].numpy()))
        label_img = label(slice, connectivity=2)
        props = regionprops(label_img)
        if len(props) < 2:
            seg_proc[i, j, :, :] = np.copy(slice)
        else:
            props.sort(key=lambda x: x.area, reverse=True)
            main_area = props[0].area
            main_centroid = props[0].centroid
            for p in props:
                distance = math.sqrt((main_centroid[0] - p.centroid[0])**2 + (main_centroid[1] - p.centroid[1])**2)
                if p.area < main_area / 8 or distance > 200 or p.area < 400:
                    for px in range(p.coords.shape[0]):
                        slice[p.coords[px, 0], p.coords[px, 1]] = False

        seg_proc[i, j, :, :] =  np.copy(slice)
        slice.astype(np.int8).tofile(seg_folder + 'processed/' + models[i] + '/output' + ("-%04d.ct" % j))

----------------- DenseNet201 -----------------
----------------- Inception -----------------
----------------- ResNet152 -----------------
----------------- ResNeXt101 -----------------
